 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#第9章:-ベクトル空間法-(I)" data-toc-modified-id="第9章:-ベクトル空間法-(I)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>第9章: ベクトル空間法 (I)</a></span><ul class="toc-item"><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#80.-コーパスの整形" data-toc-modified-id="80.-コーパスの整形-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>80. コーパスの整形</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#81.-複合語からなる国名への対処" data-toc-modified-id="81.-複合語からなる国名への対処-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>81. 複合語からなる国名への対処</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#82.-文脈の抽出" data-toc-modified-id="82.-文脈の抽出-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>82. 文脈の抽出</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#時間計測" data-toc-modified-id="時間計測-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>時間計測</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#83.-単語／文脈の頻度の計測" data-toc-modified-id="83.-単語／文脈の頻度の計測-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>83. 単語／文脈の頻度の計測</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#メモリ使用量計測" data-toc-modified-id="メモリ使用量計測-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>メモリ使用量計測</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#84.-単語文脈行列の作成" data-toc-modified-id="84.-単語文脈行列の作成-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>84. 単語文脈行列の作成</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#85.-主成分分析による次元圧縮" data-toc-modified-id="85.-主成分分析による次元圧縮-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>85. 主成分分析による次元圧縮</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#86.-単語ベクトルの表示" data-toc-modified-id="86.-単語ベクトルの表示-1.9"><span class="toc-item-num">1.9&nbsp;&nbsp;</span>86. 単語ベクトルの表示</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#87.-単語の類似度" data-toc-modified-id="87.-単語の類似度-1.10"><span class="toc-item-num">1.10&nbsp;&nbsp;</span>87. 単語の類似度</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#88.-類似度の高い単語10件" data-toc-modified-id="88.-類似度の高い単語10件-1.11"><span class="toc-item-num">1.11&nbsp;&nbsp;</span>88. 類似度の高い単語10件</a></span></li><li><span><a href="http://localhost:8889/notebooks/t-tagami/Chapter_9/Chapter_9_Server.ipynb#89.-加法構成性によるアナロジー" data-toc-modified-id="89.-加法構成性によるアナロジー-1.12"><span class="toc-item-num">1.12&nbsp;&nbsp;</span>89. 加法構成性によるアナロジー</a></span></li></ul></li></ul></div>

# 第9章: ベクトル空間法 (I)
enwiki-20150112-400-r10-105752.txt.bz2は，2015年1月12日時点の英語のWikipedia記事のうち，約400語以上で構成される記事の中から，ランダムに1/10サンプリングした105,752記事のテキストをbzip2形式で圧縮したものである．このテキストをコーパスとして，単語の意味を表すベクトル（分散表現）を学習したい．第9章の前半では，コーパスから作成した単語文脈共起行列に主成分分析を適用し，単語ベクトルを学習する過程を，いくつかの処理に分けて実装する．第9章の後半では，学習で得られた単語ベクトル（300次元）を用い，単語の類似度計算やアナロジー（類推）を行う．

なお，問題83を素直に実装すると，大量（約7GB）の主記憶が必要になる． メモリが不足する場合は，処理を工夫するか，1/100サンプリングのコーパスenwiki-20150112-400-r100-10576.txt.bz2を用いよ．

In [4]:
!wget -P data http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2

--2018-01-08 18:59:41--  http://www.cl.ecei.tohoku.ac.jp/nlp100/data/enwiki-20150112-400-r10-105752.txt.bz2
Resolving www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)... 130.34.192.83
Connecting to www.cl.ecei.tohoku.ac.jp (www.cl.ecei.tohoku.ac.jp)|130.34.192.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221903910 (212M) [application/x-bzip2]
Saving to: ‘data/enwiki-20150112-400-r10-105752.txt.bz2’

enwiki-20150112-400 100%[===================>] 211.62M  58.0MB/s    in 3.9s    

2018-01-08 18:59:47 (54.1 MB/s) - ‘data/enwiki-20150112-400-r10-105752.txt.bz2’ saved [221903910/221903910]



In [1]:
!bzcat data/enwiki-20150112-400-r10-105752.txt.bz2 | wc -l

2875326


## 80. コーパスの整形
文を単語列に変換する最も単純な方法は，空白文字で単語に区切ることである． ただ，この方法では文末のピリオドや括弧などの記号が単語に含まれてしまう． そこで，コーパスの各行のテキストを空白文字でトークンのリストに分割した後，各トークンに以下の処理を施し，単語から記号を除去せよ．
* トークンの先頭と末尾に出現する次の文字を削除: .,!?;:()[]'"
* 空文字列となったトークンは削除
以上の処理を適用した後，トークンをスペースで連結してファイルに保存せよ．

In [4]:
%%file src/80.py
import bz2
with bz2.open('data/enwiki-20150112-400-r10-105752.txt.bz2', 'rt') as f_r:
    for line in f_r:
        tokens = [word.strip(".,!?;:()[]'\"") for word in line.split()]
        print(' '.join(token for token in tokens if token))

Overwriting src/80.py


In [8]:
!python src/80.py > work/80.txt

In [9]:
!head -3 work/80.txt

Anarchism

Anarchism is a political philosophy that advocates stateless societies often defined as self-governed voluntary institutions but that several authors have defined as more specific institutions based on non-hierarchical free associations Anarchism holds the state to be undesirable unnecessary or harmful While anti-statism is central anarchism entails opposing authority or hierarchical organisation in the conduct of human relations including but not limited to the state system


## 81. 複合語からなる国名への対処
英語では，複数の語の連接が意味を成すことがある．例えば，アメリカ合衆国は"United States"，イギリスは"United Kingdom"と表現されるが，"United"や"States"，"Kingdom"という単語だけでは，指し示している概念・実体が曖昧である．そこで，コーパス中に含まれる複合語を認識し，複合語を1語として扱うことで，複合語の意味を推定したい．しかしながら，複合語を正確に認定するのは大変むずかしいので，ここでは複合語からなる国名を認定したい．

インターネット上から国名リストを各自で入手し，80のコーパス中に出現する複合語の国名に関して，スペースをアンダーバーに置換せよ．例えば，"United States"は"United_States"，"Isle of Man"は"Isle_of_Man"になるはずである．

In [10]:
!pip install pycountry

    100% |████████████████████████████████| 9.2MB 98kB/s eta 0:00:011
  Running setup.py bdist_wheel for pycountry ... done
  Stored in directory: /home/tagami/.cache/pip/wheels/95/45/76/aa36bb308428b450eb1f51955b0f26911f00df3c85602c1da8
Successfully built pycountry


In [11]:
import pycountry
country_list = [country.name for country in pycountry.countries if len(country.name.split()) > 1]
print(country_list)

['Åland Islands', 'United Arab Emirates', 'American Samoa', 'French Southern Territories', 'Antigua and Barbuda', 'Bonaire, Sint Eustatius and Saba', 'Burkina Faso', 'Bosnia and Herzegovina', 'Saint Barthélemy', 'Bolivia, Plurinational State of', 'Brunei Darussalam', 'Bouvet Island', 'Central African Republic', 'Cocos (Keeling) Islands', "Côte d'Ivoire", 'Congo, The Democratic Republic of the', 'Cook Islands', 'Cabo Verde', 'Costa Rica', 'Christmas Island', 'Cayman Islands', 'Dominican Republic', 'Western Sahara', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Micronesia, Federated States of', 'United Kingdom', 'Equatorial Guinea', 'French Guiana', 'Hong Kong', 'Heard Island and McDonald Islands', 'Isle of Man', 'British Indian Ocean Territory', 'Iran, Islamic Republic of', 'Saint Kitts and Nevis', 'Korea, Republic of', "Lao People's Democratic Republic", 'Saint Lucia', 'Sri Lanka', 'Saint Martin (French part)', 'Moldova, Republic of', 'Marshall Islands', 'Macedonia, Republic of', 'N

In [12]:
%%file src/81.py
import pycountry
country_list = [country.name for country in pycountry.countries if len(country.name.split()) > 1]

with open('work/80.txt') as f_r:
    for line in f_r:
        for country in country_list:
            if country in line:
                line = line.replace(country, country.replace(' ', '_'))
        print(line)

Writing src/81.py


In [13]:
!python src/81.py > work/81.txt

In [15]:
!grep 'United_States' work/81.txt | head -1

The term anarchism is a compound word composed from the word anarchy and the suffix -ism themselves derived respectively from the Greek i.e anarchy from anarchos meaning one without rulers from the privative prefix ἀν- an- i.e without and archos i.e leader ruler cf archon or arkhē i.e authority sovereignty realm magistracy and the suffix or -ismos -isma from the verbal infinitive suffix -ίζειν -izein The first known use of this word was in 1539."Anarchist was the term adopted by Maximilien de Robespierre to attack those on the left whom he had used for his own ends during the French Revolution but was determined to get rid of though among these anarchists there were few who exhibited the social revolt characteristics of later anarchists There would be many revolutionaries of the early nineteenth century who contributed to the anarchist doctrines of the next generation such as William Godwin and Wilhelm Weitling but they did not use the word anarchist or anarchism in describing themselv

## 82. 文脈の抽出
81で作成したコーパス中に出現するすべての単語ttに関して，単語ttと文脈語ccのペアをタブ区切り形式ですべて書き出せ．ただし，文脈語の定義は次の通りとする．

* ある単語ttの前後dd単語を文脈語ccとして抽出する（ただし，文脈語に単語ttそのものは含まない）
* 単語ttを選ぶ度に，文脈幅ddは{1,2,3,4,5}{1,2,3,4,5}の範囲でランダムに決める．

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

## 時間計測
10万行に切ったファイルで実験

In [183]:
import random
from time import time
random.seed(114514)

start = time()
with open('work/81.txt') as f_r:
    for line in f_r:
        words = line.split()
        for i, word in enumerate(words):
            window_size = random.randint(1, 5)
            contexts = words[max(i-window_size, 0):i] + words[i+1:i+window_size+1]
print('{}sec'.format(time() - start))

47.94349002838135sec


In [184]:
import random
import multiprocessing
from joblib import Parallel, delayed
random.seed(114514)

core_num = multiprocessing.cpu_count() // 2
start = time()
def proc(line):
    words = line.split()
    for i, word in enumerate(words):
        window_size = random.randint(1, 5)
        contexts = words[max(i-window_size, 0):i] + words[i+1:i+window_size+1]

with open('work/81.txt') as f_r:
    Parallel(n_jobs=core_num)(delayed(proc)(line) for line in f_r)
print('{}sec'.format(time() - start))

32.472185134887695sec


In [25]:
%%file src/82.py
import random
import multiprocessing
from joblib import Parallel, delayed
random.seed(114514)

core_num = multiprocessing.cpu_count() // 2
def proc_82(line):
    words = line.split()
    for i, word in enumerate(words):
        window_size = random.randint(1, 5)
        contexts = words[max(i-window_size, 0):i] + words[i+1:i+window_size+1]
        for context in contexts:
            print('{}\t{}'.format(word, context))

with open('work/81.txt') as f_r:
    Parallel(n_jobs=core_num)(delayed(proc_82)(line) for line in f_r)

Overwriting src/82.py


In [26]:
!python src/82.py > work/82.txt

In [27]:
!head work/82.txt

Anarchism	is
is	Anarchism
is	a
is	political
is	philosophy
a	Anarchism
a	is
a	political
a	philosophy
a	that


## 83. 単語／文脈の頻度の計測
82の出力を利用し，以下の出現分布，および定数を求めよ．

* f(t,c)f(t,c): 単語ttと文脈語ccの共起回数
* f(t,∗)f(t,∗): 単語ttの出現回数
* f(∗,c)f(∗,c): 文脈語ccの出現回数
* NN: 単語と文脈語のペアの総出現回数

## メモリ使用量計測

In [48]:
%%file src/83_memcheck1.py
from collections import defaultdict
import pickle

#愚直に数え上げる
#Counterよりdefaultdictの方が早いらしい http://hateda.hatenadiary.jp/entry/2015/03/19/223638
#10万行に切ったファイルをそのまま読み込んだ場合のメモリ使用量
@profile
def main():
    f_t = defaultdict(int)
    f_c = defaultdict(int)
    f_t_c = defaultdict(int)
    with open('work/82_.txt') as f_r:
        for line in f_r:
            word, context = line.split('\t')
            word = word.rstrip()
            context = context.rstrip()
            f_t[word] += 1
            f_c[context] += 1
            f_t_c[(word, context)] += 1
    print('N={}'.format(len(f_t_c)))

if __name__ == '__main__':
    main()

Overwriting src/83_memcheck1.py


In [49]:
!python -m memory_profiler src/83_memcheck1.py

N=72367
Filename: src/83_memcheck1.py

Line #    Mem usage    Increment   Line Contents
     4   39.652 MiB    0.000 MiB   @profile
     5                             def main():
     6   39.652 MiB    0.000 MiB       f_t = defaultdict(int)
     7   39.652 MiB    0.000 MiB       f_c = defaultdict(int)
     8   39.652 MiB    0.000 MiB       f_t_c = defaultdict(int)
     9   39.652 MiB    0.000 MiB       with open('work/82_.txt') as f_r:
    10   56.895 MiB   17.242 MiB           for line in f_r:
    11   56.895 MiB    0.000 MiB               word, context = line.split('\t')
    12   56.895 MiB    0.000 MiB               word = word.rstrip()
    13   56.895 MiB    0.000 MiB               context = context.rstrip()
    14   56.895 MiB    0.000 MiB               f_t[word] += 1
    15   56.895 MiB    0.000 MiB               f_c[context] += 1
    16   56.895 MiB    0.000 MiB               f_t_c[(word, context)] += 1
    17   56.898 MiB    0.004 MiB       print('N={}'.format(len(f_t_c)))




In [41]:
%%file src/83_memcheck2.py
from collections import defaultdict
import pickle

def chunked(iterable, n):
    return [iterable[x:x + n] for x in range(0, len(iterable), n)]

#時間を犠牲にメモリに優しく
#10万行で切ったファイルを10倍の時間をかけて読んだ場合のメモリ使用量
@profile
def main():
    f_t = defaultdict(int)
    f_c = defaultdict(int)

    with open('work/82_.txt') as f_r:
        for line in f_r:
            word, context = line.split('\t')
            word = word.rstrip()
            context = context.rstrip()
            f_t[word] += 1
            f_c[context] += 1

    word_list = [key for key in f_t.keys()]
    splited_list = chunked(word_list, len(word_list)//10)

    for splited in splited_list:
        f_t_c = defaultdict(int)
        with open('work/82_.txt') as f_r:
            for line in f_r:
                word, context = line.split('\t')
                word = word.rstrip()
                context = context.rstrip()
                if word in splited:
                    f_t_c[(word, context)] += 1

if __name__ == '__main__':
    main()

Overwriting src/83_memcheck2.py


In [42]:
!python -m memory_profiler src/83_memcheck2.py

Filename: src/83_memcheck2.py

Line #    Mem usage    Increment   Line Contents
     7   39.477 MiB    0.000 MiB   @profile
     8                             def main():
     9   39.477 MiB    0.000 MiB       f_t = defaultdict(int)
    10   39.477 MiB    0.000 MiB       f_c = defaultdict(int)
    11                             
    12   39.480 MiB    0.004 MiB       with open('work/82_.txt') as f_r:
    13   40.316 MiB    0.836 MiB           for line in f_r:
    14   40.316 MiB    0.000 MiB               word, context = line.split('\t')
    15   40.316 MiB    0.000 MiB               word = word.rstrip()
    16   40.316 MiB    0.000 MiB               context = context.rstrip()
    17   40.316 MiB    0.000 MiB               f_t[word] += 1
    18   40.316 MiB    0.000 MiB               f_c[context] += 1
    19                             
    20   40.316 MiB    0.000 MiB       word_list = [key for key in f_t.keys()]
    21   40.316 MiB    0.000 MiB       splited_list = chunked(word_list,

In [ ]:
#素直に実装してるつもりなのに余裕で7GB超える 岡崎さんの素直[要出典]とは
#時期が悪すぎてサーバー激混み
from collections import defaultdict
import pickle

f_t = defaultdict(int)
f_c = defaultdict(int)
f_t_c = defaultdict(int)

with open('work/82.txt') as f_r:
    for line in f_r:
        word, context = line.split('\t')
        word = word.rstrip()
        context = context.rstrip()
        f_t[word] += 1
        f_c[context] += 1
        f_t_c[(word, context)] += 1

with open('work/f_t.pickle', 'wb') as f, open('work/f_c.pickle', 'wb') as g, open('work/f_t_c.pickle', 'wb') as h:
    pickle.dump(f_t, f)
    pickle.dump(f_c, g)
    pickle.dump(f_t_c, h)
with open('work/N.pickle', 'wb') as i:
    pickle.dump(len(f_t_c), i)

# with open('work/f_t.txt', 'w') as f, open('work/f_c.txt', 'w') as g, open('work/f_t_c.txt', 'w') as h:
#     for i, j in f_t.items():
#         f.write('{}\t{}\n'.format(i, j))
#     for i, j in f_c.items():
#         g.write('{}\t{}\n'.format(i, j))
#     for i, j in f_t_c.items():
#         h.write('{}\t{}\n'.format(i, j))

N=129320712


In [41]:
#並列にしてるはずがめちゃくちゃ遅くなる 辛い
from collections import defaultdict
from joblib import Parallel, delayed
import pickle

f_t = defaultdict(int)
f_c = defaultdict(int)
f_t_c = defaultdict(int)

core_num = multiprocessing.cpu_count() // 2

def proc_83(line):
    word, context = line.split('\t')
    word = word.rstrip()
    context = context.rstrip()
    f_t[word] += 1
    f_c[context] += 1
    f_t_c[(word, context)] += 1

with open('work/82.txt') as f_r:
    Parallel(n_jobs=core_num)(delayed(proc_83)(line) for line in f_r)
        
with open('work/f_t.pickle', 'wb') as f, open('work/f_c.pickle', 'wb') as g, open('work/f_t_c.pickle', 'wb') as h:
    pickle.dump(f_t, f)
    pickle.dump(f_c, g)
    pickle.dump(f_t_c, h)
with open('work/N.pickle', 'wb') as i:
    pickle.dump(len(f_t_c), i)
        
# with open('work/f_t.txt', 'w') as f, open('work/f_c.txt', 'w') as g, open('work/f_t_c.txt', 'w') as h:
#     for i, j in f_t.items():
#         f.write('{}\t{}\n'.format(i, j))
#     for i, j in f_c.items():
#         g.write('{}\t{}\n'.format(i, j))
#     for i, j in f_t_c.items():
#         h.write('{}\t{}\n'.format(i, j))

Process ForkPoolWorker-12:
KeyboardInterrupt
Process ForkPoolWorker-8:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-11:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/multiprocessing/process.py", line 249, in

KeyboardInterrupt: 

## 84. 単語文脈行列の作成
83の出力を利用し，単語文脈行列XXを作成せよ．ただし，行列XXの各要素XtcXtcは次のように定義する．

* f(t,c)≥10f(t,c)≥10ならば，Xtc=PPMI(t,c)=max{logN×f(t,c)f(t,∗)×f(∗,c),0}Xtc=PPMI(t,c)=max{log⁡N×f(t,c)f(t,∗)×f(∗,c),0}
* f(t,c)<10f(t,c)<10ならば，Xtc=0Xtc=0
ここで，PPMI(t,c)PPMI(t,c)はPositive Pointwise Mutual Information（正の相互情報量）と呼ばれる統計量である．なお，行列XXの行数・列数は数百万オーダとなり，行列のすべての要素を主記憶上に載せることは無理なので注意すること．幸い，行列XXのほとんどの要素は00になるので，非00の要素だけを書き出せばよい．

In [1]:
import pickle
import numpy as np
import scipy.sparse as sp

with open('work/f_t.pickle', mode='rb') as f:
    f_t = pickle.load(f)
with open('work/f_c.pickle', mode='rb') as g:
    f_c = pickle.load(g)
with open('work/f_t_c.pickle', mode='rb') as h:
    f_t_c = pickle.load(h)
with open('work/N.pickle', mode='rb') as i:
    N = pickle.load(i)
    
idx = {word:num for num, word in enumerate(f_t.keys())}
idx_size = len(idx)
f_matrix = sp.lil_matrix((idx_size, idx_size))

for t_c, f_t_c_num in f_t_c.items():
    if f_t_c_num > 9:
        f_matrix[idx[t_c[0]], idx[t_c[1]]] = max(np.log10(N*f_t_c_num*1.0 / (f_c[t_c[1]] * f_t[t_c[0]])*1.0), 0)
    
with open('work/84.pickle', 'wb') as f:
    pickle.dump(f_matrix, f)

In [2]:
f_matrix.nonzero

<bound method spmatrix.nonzero of <1808218x1808218 sparse matrix of type '<class 'numpy.float64'>'
	with 1231308 stored elements in LInked List format>>

## 85. 主成分分析による次元圧縮
84で得られた単語文脈行列に対して，主成分分析を適用し，単語の意味ベクトルを300次元に圧縮せよ．

In [3]:
#http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html
from sklearn.decomposition import TruncatedSVD
import pickle

with open('work/84.pickle', mode = 'rb') as f, open('work/85.pickle', 'wb') as g:
    compressed_matrix = TruncatedSVD(300).fit_transform(pickle.load(f))
    pickle.dump(compressed_matrix, g, protocol=4)

## 86. 単語ベクトルの表示
85で得た単語の意味ベクトルを読み込み，"United States"のベクトルを表示せよ．ただし，"United States"は内部的には"United_States"と表現されていることに注意せよ．

In [4]:
with open('work/85.pickle', mode = 'rb') as f, open('work/f_t.pickle', mode='rb') as g:
    compressed_matrix = pickle.load(f)
    f_t = pickle.load(g)
    idx = {word:num for num, word in enumerate(f_t.keys())}
    
def word_to_vec(word):
    if word in idx:
        return(compressed_matrix[idx[word]])

In [212]:
word_to_vec('United_States')

array([  2.22608626e-01,   1.33169412e+00,  -2.91147759e-01,
        -5.97374155e-02,   2.48949822e-01,  -2.87291890e-01,
         4.50835228e-01,  -2.30375890e-01,  -4.88754087e-02,
        -5.83240878e-01,   1.98315105e-01,  -8.47092755e-03,
        -8.96536295e-01,  -6.04573734e-02,  -8.74321141e-01,
         3.56227635e-01,  -1.16886495e+00,  -1.40522692e-01,
         6.35137278e-01,   9.92096985e-02,   4.01139757e-01,
        -4.67382090e-01,  -4.79606936e-01,   5.78501914e-02,
        -7.41588521e-03,   1.60342639e-01,   1.90910175e-01,
        -3.97434763e-02,   1.45836810e-01,   8.94296560e-02,
        -3.05635787e-01,   6.83708832e-02,   5.62568507e-02,
        -1.14634595e-01,  -1.46208405e-01,  -9.55660159e-02,
        -7.84077407e-02,   5.19851641e-02,  -1.67811956e-01,
         2.05772023e-01,   2.62542009e-01,   2.07036984e-03,
         3.34689546e-02,  -1.32484455e-01,   1.96644234e-01,
         1.54922464e-01,   2.89121211e-01,  -2.19886982e-01,
         2.29096847e-01,

## 87. 単語の類似度
85で得た単語の意味ベクトルを読み込み，"United States"と"U.S."のコサイン類似度を計算せよ．ただし，"U.S."は内部的に"U.S"と表現されていることに注意せよ．

In [5]:
#from sklearn.metrics.pairwise import cosine_similarity こっちだと2D array云々でエラーでてめんどい
import scipy.spatial.distance as scd
def calc_cos_similarity(word1, word2):
    vec1 = word_to_vec(word1)
    vec2 = word_to_vec(word2)
    if vec1 is not None and vec2 is not None:
        return(1 - scd.cosine(vec1, vec2))

In [6]:
calc_cos_similarity('United_States', 'U.S')

0.81166901944255931

## 88. 類似度の高い単語10件
85で得た単語の意味ベクトルを読み込み，"England"とコサイン類似度が高い10語と，その類似度を出力せよ．

In [7]:
from pprint import pprint

def get_top10_similar_words(word):
    result = []
    for word1 in idx.keys():
        cos_sim = calc_cos_similarity(word, word1)
        if cos_sim > 0 and word1 != word:
            result.append([word1, cos_sim])
    result.sort(key=lambda x:x[1], reverse = True)
    pprint(result[:10])

In [8]:
get_top10_similar_words('England')

/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


[['selectors', 0.82924151040292426],
 ['Herefordshire', 0.82473426808423278],
 ['Cumbria', 0.77641631845756975],
 ['Devizes', 0.74908334645347308],
 ['Quays', 0.74539005033160421],
 ['Ashbourne', 0.73946964355800593],
 ['Normans', 0.73607639443364592],
 ['Wolds', 0.73335782919381187],
 ['Woking', 0.7302595609901853],
 ['Huntingdonshire', 0.72756787628291919]]


## 89. 加法構成性によるアナロジー
85で得た単語の意味ベクトルを読み込み，vec("Spain") - vec("Madrid") + vec("Athens")を計算し，そのベクトルと類似度の高い10語とその類似度を出力せよ．

In [9]:
#from sklearn.metrics.pairwise import cosine_similarity こっちだと2D array云々でエラーでてめんどい
import scipy.spatial.distance as scd
def calc_cos_similarity_(vec, word):
    vec1 = word_to_vec(word)
    if vec is not None and vec1 is not None:
        return(1 - scd.cosine(vec, vec1))

In [10]:
from pprint import pprint

def get_top10_similar_words(word):
    result = []
    for word1 in idx.keys():
        cos_sim = calc_cos_similarity_(word, word1)
        if cos_sim > 0 and word1 != word:
            result.append([word1, cos_sim])
    result.sort(key=lambda x:x[1], reverse = True)
    pprint(result[:10])

In [11]:
vec = word_to_vec('Spain') - word_to_vec('Madrid') + word_to_vec('Athens')
get_top10_similar_words(vec)

/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys
/home/tagami/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/scipy/spatial/distance.py:505: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v))


[['Spain', 0.84438047910663439],
 ['Portugal', 0.80572639343589325],
 ['Denmark', 0.79830433842360948],
 ['Grandee', 0.79705620741072691],
 ['Malaga', 0.79704842007566534],
 ['Cadiz', 0.79704842007566523],
 ['Andalusia', 0.79704842007534704],
 ['Málaga', 0.79465967720933017],
 ['Belgium', 0.79115157348938558],
 ['Sweden', 0.78811707862648916]]
